In [22]:
import networkx
import math
import random
import copy
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm 
import pickle

In [23]:
Graph_2 = networkx.read_gpickle("Graph2")

In [24]:
dict_bfs_tree = {}
dict_distances = {}
dict_shortest_path_counts = {}
dict_predecessors = {}
list_nodes = list(Graph_2.nodes())
len_list_nodes = len(list_nodes)
#First initilaizing the dictionaries for different purpose
for a in list_nodes:
    dict_bfs_tree[a] = []
    dict_distances[a] = []
    dict_shortest_path_counts[a] = []
    dict_predecessors[a] = []

dict_dict_parents = {}
for a in list_nodes:
    dict_dict_parents[a] = copy.deepcopy(dict_predecessors)
    
val_infinity = math.inf
for i in range(len_list_nodes):
    curr_node = list_nodes[i]
    temp = [val_infinity for j in range(len_list_nodes)]
    dict_distances[curr_node] = temp
    dict_distances[curr_node][curr_node] = 0
    dict_shortest_path_counts[curr_node] = [0 for k in range(len_list_nodes)]
    dict_shortest_path_counts[curr_node][curr_node] = 1
    
for v in list_nodes:
    Graph_bfs = networkx.DiGraph()
    for ip in range(len_list_nodes):
        Graph_bfs.add_node(ip)
    list_visited = [0 for i in range(len_list_nodes)]
    bfs_queue = []
    list_visited[v] = 1
    bfs_queue.append(v)
    
    dict_prev = {}
    list_source_distances = dict_distances[v]
    list_source_counts = dict_shortest_path_counts[v]
    while len(bfs_queue) > 0:
        curr_node = bfs_queue.pop(0)
        list_neighbours = [x for x in Graph_2.neighbors(curr_node)]
        for nd in list_neighbours:
            if list_visited[nd] == 0:
                list_visited[nd] = 1
                bfs_queue.append(nd)
                Graph_bfs.add_edge(curr_node, nd)

            if list_source_distances[nd] > list_source_distances[curr_node] + 1:
                list_source_distances[nd] = list_source_distances[curr_node] + 1
                list_source_counts[nd] = list_source_counts[curr_node]
                dict_dict_parents[v][nd] = [curr_node]
            
            elif list_source_distances[nd] == list_source_distances[curr_node] + 1:
                list_source_counts[nd] = list_source_counts[nd] + list_source_counts[curr_node]
                dict_dict_parents[v][nd].append(curr_node)

    dict_distances[v] = list_source_distances
    dict_shortest_path_counts[v] = list_source_counts
    dict_bfs_tree[v] = Graph_bfs

In [25]:

dict_dependency_values = {}
dict_dependency = {}
edge_list = Graph_2.edges()
counter_edges = Counter(edge_list)

In [26]:
def find_dependency(src, edg):
    u = edg[0]
    v = edg[1]
    distance_to_u = dict_distances[src][u]
    distance_to_v = dict_distances[src][v]
    if distance_to_u > distance_to_v:
        temp_store = u
        u = v
        v = temp_store
    value_dependency = dict_shortest_path_counts[src][u] / dict_shortest_path_counts[src][v]
    if current_node_bfs_tree.out_degree(v) == 0:
        if counter_edges[edg] == 1:
            dict_dependency[src][edg] = value_dependency
            return value_dependency
        else:    
            return value_dependency

    else:
        #Now find vertices which containg vertex v in their parent set
        list_x_nodes = []
        dict_curr_parents = dict_dict_parents[src]
        for aq in dict_curr_parents:
            list_parents = dict_curr_parents[aq]
            if v in list_parents:
                list_x_nodes.append(aq)

        #Finding the value of summation
        value_summation = 0
        for x in list_x_nodes:
            req_ed = (v, x)
            if dict_dependency[src].get(req_ed) is None:
                value_summation += find_dependency(src, req_ed)
            else:    
                value_summation += dict_dependency[src][req_ed]
        final_val = value_dependency * (1 + value_summation)
        if counter_edges[edg] == 1:
            dict_dependency[src][edg] = final_val
        else:
            return final_val
        
#Now finding the dependency of each vertex on each of the edges in the graph
list_edges = list(Graph_2.edges())
for s in tqdm(list_nodes):
    current_node_bfs_tree = dict_bfs_tree[s]
    dict_dependency[s] = {}
    
    for e in list_edges:
        if dict_dependency[s].get(e) is None:
            dict_dependency[s][e] =  find_dependency(s, e)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:49<00:00,  2.28it/s]


In [27]:
#Now finding the final edge betweeness value for each edge
dict_edge_betweenness = {}
for e in list_edges:
    value_betweenness = 0
    for n in dict_dependency:
        value_betweenness += dict_dependency[n][e]
    dict_edge_betweenness[e] = value_betweenness

In [28]:
# print(dict_edge_betweenness)

In [29]:
Picklefile1 = open('dictionary_betweenness_algo1', 'wb')
pickle.dump(dict_edge_betweenness, Picklefile1)
Picklefile1.close()
print("Dictionary pic")
